In [172]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.linalg import cho_solve, cho_factor
from tqdm import tqdm

np.set_printoptions(precision=4, linewidth=200, sign=" ")

## Useful functions from pose_graph_jacobian.ipynb
Just functions for `Exp` and `Log` maps, `cross` product matrix function and calculation of the jacobian

In [173]:
def np_cross(x):
    return np.array([
        [0,-x[2],x[1]],
        [x[2],0,-x[0]],
        [-x[1],x[0],0]
    ])

def np_exp(x):
    theta = np.linalg.norm(x[3:])
    if theta == 0:
        R = np.eye(3)
        t = x[:3]
    else:
        a = np.sin(theta)/theta
        b = (1 - np.cos(theta))/(theta*theta)
        c = (theta - np.sin(theta))/(theta**3)
        w = np_cross(x[3:])
        R = np.eye(3) + a*w + b*w@w

        t = (np.eye(3) + b*w + c*w@w)@x[:3]
    
    mat = np.eye(4)
    mat[:3, :3] = R
    mat[:3, 3] = t

    return mat

def np_log(T):
    theta = np.arccos((np.trace(T) - 2) / 2)
    # theta = np.arccos((T[0, 0] + T[1, 1] + T[2, 2] - 1) / 2)

    if theta == 0:
        return np.array([T[0, 3], T[1, 3], T[2, 3], 0, 0, 0
])
    a = np.sin(theta)/theta
    b = (1 - np.cos(theta))/(theta*theta)

    x4 = 1/(2*a)*(T[2, 1] - T[1, 2])
    x5 = 1/(2*a)*(T[0, 2] - T[2, 0])
    x6 = 1/(2*a)*(T[1, 0] - T[0, 1])
    
    w = np_cross(np.array([x4,x5,x6]))
    u = (np.eye(3) - 1/2*w + 1/(theta**2)*(1 - a/(2*b))*w@w)@T[:3, 3]

    return np.array([u[0], u[1], u[2], x4, x5, x6])

In [174]:
def np_adjoint(X):
    """X is a 4x4 matrix not a 6 Vector"""
    mat = np.zeros((6,6))
    mat[:3,:3] = X[:3,:3]
    mat[3:,3:] = X[:3,:3]
    mat[:3,3:] = np_cross(X[:3,3])@X[:3,:3]

    return mat.T

def np_error(Z, Xi, Xj):
    return np_log(np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj))

def np_exp_error(Z, Xi, Xj):
    return np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj)

In [175]:
def box_plus(a, b):
    return np_log(np_exp(a)@np_exp(b))

def box_minus(a, b):
    return np_log(np.linalg.inv(np_exp(b))@np_exp(a))

In [176]:
def Je_analytic_Xi(Z, Xi, Xj):
    eXi, eXj = np_exp(Xi), np_exp(Xj)
    A = np.zeros((6,6))
    A[:3,:3] = (eXj[:3,:3]).T
    A[3:,3:] = (eXj[:3,:3]).T
    A[:3,3:] = -(eXj[:3,:3]).T@np_cross(eXj[:3,3])

    B = np.zeros_like(A)
    B[:3, :3] = eXi[:3,:3]
    B[3:,3:] = eXi[:3,:3]
    B[:3,3:] = np_cross(eXi[:3, 3])@eXi[:3,:3]

    return -A@B

def Je_analytic_Xj(Z, Xi, Xj):
    return np.eye(6)

In [177]:
def Je_numeric_Xi(Z, Xi, Xj, eps=1E-8, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error = np_exp_error(Z, Xi, Xj)
    # error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    # gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi)@np_exp(g_i*eps))@np_exp(Xj) for g_i in generators]
    gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi + g_i*eps))@np_exp(Xj) for g_i in generators]
    # gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi + g_i*eps))@np_exp(Xj) for g_i in generators]

    # cols = (1/eps)*np.array([np_log(error_inv@gen_error_i) for gen_error_i in gen_error]).T
    try:
        cols = (1/eps)*np.array([np_log(gen_error_i) - np_log(error) for gen_error_i in gen_error]).T
    except TypeError:
        print(gen_error)
        print(error)
        raise

    return cols


def Je_numeric_Xj(Z, Xi, Xj, eps=1E-8, type_=np.float64):    
    Z, Xi, Xj = Z.astype(type_), Xi.astype(type_), Xj.astype(type_)

    generators = [np.array([i==j for i in range(6)], dtype=type_) for j in range(6)]

    error = np_exp_error(Z, Xi, Xj)
    # error_inv = np.linalg.inv(np_exp_error(Z, Xi, Xj))
    # gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@(np_exp(Xj)@np_exp(g_i*eps))for g_i in generators]
    gen_error = [np.linalg.inv(np_exp(Z))@np.linalg.inv(np_exp(Xi))@np_exp(Xj + g_i*eps) for g_i in generators]


    # cols = (1/eps)*np.array([np_log(error_inv@gen_error_i) for gen_error_i in gen_error]).T
    cols = (1/eps)*np.array([np_log(gen_error_i) - np_log(error) for gen_error_i in gen_error]).T

    return cols

## Setting up the pose graph problem
The plan here is to define a pose graph problem that is simple to solve and has a known correct solution - i.e. I can check that I'm converging on the right spot. To start with I think I'm going to go with 3 fully connected poses. I then calculate the correct measurements between them as my measurements, and set the information matrices to something reasonable. After that, displace the points and use these new points as my starting off point for the optimisation - if it correctly comes back to the original position then I'll know I've done something right and can move onto progressively harder problems.

In [178]:
n = 50
poses = [np_log(np_exp((np.random.random(6) - 0.5)*2)) for _ in range(n)]
measurements = {(i,j):np_log(np.linalg.inv(np_exp(poses[i]))@np_exp(poses[j])) for (i, j) in combinations(range(n), 2)}

eps = 0.1
disturbed_poses = [p + np.random.normal(0, eps, 6) if i else p for i, p in enumerate(poses)]
disturbed_poses[0] = poses[0]

infos = {k:np.linalg.inv(np.eye(6)*0.05) for k in measurements.keys()}

# disturbed_poses, poses

### Take new measurements
So we are going to take all the consecutive measurements and then a few "loop closure" measurements and see what happens

In [179]:
new_measures = {}
for i in range(n - 1):
    # print(measurements[(i,i+1)])
    new_measures[(i, i + 1)] = measurements.pop((i,i+1))

keys = tuple(measurements.keys())
for i in np.random.choice(len(keys), 2):
    new_measures[keys[i]] = measurements[keys[i]]

measurements = new_measures
len(new_measures)

51

Now that we have our poses, measurements and information matrices its time to get on with writing the actual optimisation code. We're going to use Gauss-Newton I think so the update rule is going to look like this:

$$\Delta x = (J^TJ)^{-1}J^Tr$$

In [180]:
def make_parts(poses, measurements, infos):
    H = np.zeros((6*len(poses), 6*len(poses)))
    b = np.zeros(6*len(poses))

    H[:6, :6] += np.eye(6)

    for (i, j), Zij in measurements.items():
        Xi, Xj = poses[i], poses[j]
        omega = infos[(i, j)]

        e = np_error(Zij, Xi, Xj)
        A = Je_numeric_Xi(Zij, Xi, Xj)
        B = Je_numeric_Xj(Zij, Xi, Xj)

        H[6*i:6*i+6, 6*i:6*i+6] += A.T@omega@A
        H[6*i:6*i+6, 6*j:6*j+6] += A.T@omega@B
        H[6*j:6*j+6, 6*i:6*i+6] += B.T@omega@A
        H[6*j:6*j+6, 6*j:6*j+6] += B.T@omega@B

        b[6*i:6*i+6] += A.T@omega@e
        b[6*j:6*j+6] += B.T@omega@e

    return H, b

H, b = make_parts(disturbed_poses, measurements, infos)

H.shape, b.shape


((300, 300), (300,))

In [187]:
new_poses = disturbed_poses[:]
print([p for p in new_poses])

for i in tqdm(range(10)):
    H, b = make_parts(new_poses, measurements, infos)
    # with np.printoptions(precision=4, suppress=True):
    #     print(H)
    delta = cho_solve(cho_factor(H), b)

    pose_vector = np.concatenate(new_poses)
    for i in range(0, pose_vector.shape[0]//6):
        # new_poses[i] = np_log(np_exp(new_poses[i])@np_exp(delta[6*i:6*i + 6]))
        new_poses[i] = np_log(np.linalg.inv(np_exp(delta[6*i:6*i + 6]))@np_exp(new_poses[i]))
        # new_poses[i] += delta[6*i:6*i+6]
    # print([p for p in new_poses])


# print(new_poses)
# print(poses)
    

[array([-0.074 , -0.1974, -0.7999,  0.9403, -0.3081,  0.0518]), array([-0.998 ,  0.6752,  0.5913,  0.7361, -0.5811,  0.1681]), array([ 1.0657e+00, -5.7989e-01, -5.5361e-01,  9.1077e-01,  7.7744e-04, -7.8859e-01]), array([ 0.7425,  0.967 ,  0.4661, -0.7872, -0.8855, -0.7158]), array([-1.0983, -0.4692, -0.7568, -0.5052,  0.0334, -0.5942]), array([ 0.274 ,  0.2962, -0.4586,  0.7342, -0.5818, -0.9403]), array([-0.874 , -0.5619,  0.4137,  0.2611,  0.3736, -0.5892]), array([-1.0508, -0.6791,  1.0318,  0.3624, -0.5981,  0.2431]), array([ 0.6245, -0.5823,  0.5267, -0.7601,  0.5937, -0.29  ]), array([ 0.1125,  0.4108,  0.7289,  0.906 , -0.3417,  0.0766]), array([ 0.8554, -0.0372,  0.2734, -0.1757, -0.1841,  0.6898]), array([ 0.8793, -0.112 , -0.8183,  0.9917, -0.2922, -0.4234]), array([ 0.0409, -0.4232, -0.8385,  0.3685, -0.7888, -0.0242]), array([-0.9908, -0.5261, -0.0381, -0.2667,  0.0831,  0.7216]), array([-0.9311, -0.205 ,  0.2429,  0.1843, -0.7617, -0.0327]), array([-0.039 ,  0.4552, -1.03

100%|██████████| 6/6 [00:00<00:00, 16.10it/s]


## Visualising the Optimisation
The plan is to visualise the optimisation process using open3d and some coloured dots.
- Red is the initial "bad" position of the 

In [188]:
import open3d as o3d
from itertools import chain

In [189]:
def create_axis(transform, colour, size=0.2):
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size, origin=np.array([0.0,0.0,0.0]))
    mesh.paint_uniform_color(colour)
    mesh.transform(transform)
    return mesh

In [190]:
true_poses = [create_axis(np_exp(x), [0,1,0]) for x in poses]
start_poses = [create_axis(np_exp(x), [1,0,0]) for x in disturbed_poses]
end_poses = [create_axis(np_exp(x), [0,0,1], 0.3) for x in new_poses]

# o3d.visualization.draw_geometries([x for x in chain(start_poses, true_poses, end_poses)])

In [191]:
def create_error(start, error, colour, radius=0.005, resolution=20):
    end = start@error
    # Extract translation vectors from the matrices
    start_translation = np.array(start[:3, 3])
    end_translation = np.array(end[:3, 3])

    # Calculate the height of the cylinder
    height = np.linalg.norm(end_translation - start_translation)

    # Create a cylinder
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=resolution)

    # Calculate the rotation matrix to align the cylinder with the direction from start to end
    z = [0,0,1]
    direction = end_translation - start_translation
    dz = direction/np.linalg.norm(direction)
    dx = np.cross(dz, z)/np.linalg.norm(np.cross(dz, z))
    dy = np.cross(dz, dx)/np.linalg.norm(np.cross(dz, dx))

    rotation_matrix = np.vstack([dx,dy,dz]).T
    print(rotation_matrix)
    print(dx, dy, dz)

    # Apply the rotation and translation to the cylinder
    cylinder.translate(np.array([0, 0, height/2]))
    cylinder.rotate(rotation_matrix, center=(0,0,0))
    cylinder.translate(start_translation)
    cylinder.paint_uniform_color(colour)

    return cylinder

errors = [create_error(np_exp(poses[i]), np_exp(Zij), [0.7,0.7,0.7]) for (i, j), Zij in measurements.items()]

o3d.visualization.draw_geometries([x for x in chain(errors, start_poses, end_poses, true_poses, [create_axis(np.eye(4), [0,0,0])])])

[[ 0.1875 -0.636  -0.7486]
 [ 0.9823  0.1214  0.1429]
 [-0.     -0.7621  0.6474]]
[ 0.1875  0.9823 -0.    ] [-0.636   0.1214 -0.7621] [-0.7486  0.1429  0.6474]
[[-0.494  -0.4036  0.7701]
 [-0.8695  0.2293 -0.4375]
 [ 0.     -0.8857 -0.4642]]
[-0.494  -0.8695  0.    ] [-0.4036  0.2293 -0.8857] [ 0.7701 -0.4375 -0.4642]
[[ 0.9933  0.0756  0.0874]
 [-0.1156  0.6499  0.7512]
 [ 0.     -0.7562  0.6543]]
[ 0.9933 -0.1156  0.    ] [ 0.0756  0.6499 -0.7562] [ 0.0874  0.7512  0.6543]
[[-0.4595  0.3795 -0.803 ]
 [ 0.8882  0.1963 -0.4155]
 [ 0.     -0.9041 -0.4273]]
[-0.4595  0.8882  0.    ] [ 0.3795  0.1963 -0.9041] [-0.803  -0.4155 -0.4273]
[[ 0.3203  0.1361  0.9375]
 [-0.9473  0.046   0.3169]
 [ 0.     -0.9896  0.1437]]
[ 0.3203 -0.9473  0.    ] [ 0.1361  0.046  -0.9896] [ 0.9375  0.3169  0.1437]
[[-0.5467 -0.4532 -0.7041]
 [ 0.8373 -0.2959 -0.4597]
 [ 0.     -0.8408  0.5413]]
[-0.5467  0.8373  0.    ] [-0.4532 -0.2959 -0.8408] [-0.7041 -0.4597  0.5413]
[[-0.9431  0.1386 -0.3022]
 [ 0.3325  0.